In [1]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm
import xgboost
import sklearn
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import class_weight

/Users/tdao/opt/anaconda3/envs/pyaml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df_X = pd.read_csv("data/X_train_features_final.csv", index_col=0)
df_Y = pd.read_csv("data/y_train.csv", index_col="id")

In [3]:
from sklearn.model_selection import RandomizedSearchCV

lr = [0.05, 0.1, 0.2]
n_estimators = [500, 600, 700]
max_depth = [6, 7, 8, 9]

random_grid = {'learning_rate': lr,
               'n_estimators': n_estimators,
               'max_depth': max_depth}
print(random_grid)

{'learning_rate': [0.05, 0.1, 0.2], 'n_estimators': [500, 600, 700], 'max_depth': [6, 7, 8, 9]}


In [4]:
xgb = xgboost.XGBClassifier(use_label_encoder=False)
xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [5]:
X = df_X.to_numpy()
Y = df_Y.to_numpy().squeeze()
xgb_random.fit(X, Y)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/Users/tdao/opt/anaconda3/envs/pyaml/lib/python3.10/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 36 is smaller than n_iter=100. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/tdao/opt/anaconda3/envs/pyaml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/tdao/opt/anaconda3/envs/pyaml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/tdao/opt/anaconda3/envs/pyaml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas 

[12:30:29] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:30:29] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:30:29] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints...
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None,
                                           reg_alpha=None, reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=None, tree_method=None,
                                           use_label_encoder=False,
                                           validate_parameters=None,
                                           verbosity=None),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.1, 0.2],
                                        'max_depth': [6, 7, 8, 9],
                                        'n_estimators': [500, 600, 700]},
                   random_state=42, verbose=2)

In [6]:
xgb_random.best_params_

{'n_estimators': 700, 'max_depth': 8, 'learning_rate': 0.05}

In [7]:
# best params
best_params = {'n_estimators': 700, 
               'max_depth': 8, 
               'learning_rate': 0.05}

Evaluate

In [10]:
skf = StratifiedKFold(n_splits=5)
f1_base = []
f1_optim = []

for train_index, test_index in skf.split(X, Y):
    clf = xgboost.XGBClassifier(use_label_encoder=False)
    
    clf.fit(X[train_index], Y[train_index])
    y_pred = clf.predict(X[test_index])
    f1_base.append(f1_score(Y[test_index], y_pred, average='micro'))
    
    clf_optim = xgboost.XGBClassifier(use_label_encoder=False,n_estimators=700, max_depth=8, learning_rate=0.05)
    clf_optim.fit(X[train_index], Y[train_index])
    y_pred = clf_optim.predict(X[test_index])
    f1_optim.append(f1_score(Y[test_index], y_pred, average='micro'))

[14:13:45] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:13:49] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:14:19] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to

In [11]:
# knn
print(np.mean(f1_base))
print(np.mean(f1_optim))

0.8516713251466277
0.8544085639662757
